In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
import pandas as pd

df = pd.read_csv('/content/drive/MyDrive/Thesis/posts1352_standard_representations.csv')

In [ ]:
df

,No,Category,SubCategory,Post,HateSpeech,PreprocessedToken,Topic_Probabilities,Topic,Representative_Docs
0,1,Race,Chinese,Chinese are pigs!,1,"['chinese', 'pigs']","[4, 0.353802893618629]","['chinese', 'china', 'people', 'pig', 'go', 'i...","['chinese pig', 'pig chinese', 'chinese people..."
1,2,Race,Indian,I don't like pajeets they smell like curry,1,"['unalike', 'pajeets', 'smell', 'like', 'curry']","[7, 0.21967016813460452]","['indians', 'indian', 'pajeets', 'smell', 'paj...","['like indians', 'indians', 'indians']"
2,4,Race,Indian,"Indians always smell stinky and weird, what is...",1,"['indians', 'smell', 'stinky', 'weird']","[7, 0.20338674496166917]","['indians', 'indian', 'pajeets', 'smell', 'paj...","['like indians', 'indians', 'indians']"
3,5,Gender,Female,Women are so emotional they can't handle anyth...,1,"['women', 'emotional', 'handle', 'anything', '...","[1, 1.0]","['men', 'women', 'woman', 'man', 'sex', 'want'...","['american women literally men', 'men', 'women..."
4,6,Race,Malay,Malays are dependant on subsidiaries from the ...,1,"['malays', 'dependant', 'subsidiaries', 'gover...","[2, 1.0]","['malay', 'malays', 'malaysia', 'malaysian', '...","['malays chinese', 'malaysian malay', 'malay']"
...,...,...,...,...,...,...,...,...,...
1615,1349,Religion,Buddhism,You cannot build a temple in an area with majo...,1,"['build', 'temple', 'area', 'majority', 'musli...","[0, 0.0]","['disgusting', 'like', 'people', 'middle', 'sy...","['well idk rest never encountered problem', 'w..."
1616,1350,Religion,Buddhism,"Muslims should not enter temple, or they might...",1,"['muslims', 'exit', 'temple', 'might', 'accide...","[14, 1.0]","['malaysia', 'buddhists', 'muslims', 'would', ...",['expel buddhists hindus christians malaysia n...
1617,1351,Gender,Male,Men that don't have job are useless piece of s...,1,"['men', 'lack', 'job', 'useless', 'piece', 'sh...","[1, 1.0]","['men', 'women', 'woman', 'man', 'sex', 'want'...","['american women literally men', 'men', 'women..."
1618,1352,Gender,Female,Women are dick machines.,1,"['women', 'dick', 'machines']","[1, 1.0]","['men', 'women', 'woman', 'man', 'sex', 'want'...","['american women literally men', 'men', 'women..."


In [114]:
import gensim.corpora as corpora
import gensim.models as models

from ast import literal_eval
from pprint import pprint

bertopic_words = []
y_list = []

# for x in df['Topic']:
# for x in df['Topic_Probabilities']:
# for x in df['Representative_Docs']:
# for x in df['PreprocessedToken']:
for x in df['Representative_Docs']:
    bertopic_words.append(x)

bertopic_words = bertopic_words[:1600]

for y in df['HateSpeech']:
    y_list.append(y)

y_list = y_list[:1600]

In [115]:
# obtain token list that's actually list instead of string (ignore if it's topic probabilities)
from ast import literal_eval

temp = []

for x in bertopic_words:
    temp.append((literal_eval(x)))

bertopic_words = temp

In [116]:
# Flatten the list to find unique strings
flattened = [item for sublist in bertopic_words for item in sublist]

# Create a mapping of each unique string to a unique integer
unique_strings = list(set(flattened))
string_to_int = {string: i for i, string in enumerate(unique_strings)}

# Transform the original list using the mapping
converted_list = [[string_to_int[item] for item in sublist] for sublist in bertopic_words]

# Normalize the converted list
min_val = min(min(sublist) for sublist in converted_list)
max_val = max(max(sublist) for sublist in converted_list)

normalized_list = [[(item - min_val) / (max_val - min_val) for item in sublist] for sublist in converted_list]

Don't run this if it's PreprocessedToken

In [117]:
#change into tuple

data = []
for i in range(len(normalized_list)):
  temp_tuple = (normalized_list[i],y_list[i])
  data.append(temp_tuple)

This section is for PreprocessedToken only

In [97]:
# Pad the sequences to have the same length ()
from tensorflow.keras.preprocessing.sequence import pad_sequences

max_sequence_length = 46
padded_data = pad_sequences(converted_list, maxlen=max_sequence_length)
#data = np.array(data)
#labels = np.array(labels)

In [98]:
#change into tuple

data = []
for i in range(len(converted_list)):
  temp_tuple = (padded_data[i],y_list[i])
  data.append(temp_tuple)

Continue back to the public section

In [118]:
num_epochs = 10
batch_size = 32
learning_rate = 0.001

In [119]:
from torch.utils.data import Dataset, DataLoader

class CustomDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        x, y = self.data[idx]
        return torch.tensor(x, dtype=torch.float32), torch.tensor(y, dtype=torch.float32)

dataset = CustomDataset(data)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [110]:
#Topic

class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.conv1 = nn.Conv1d(1, 4, 3) # default stride = 1
        # channels of input, channels of output, filter size: 3 x 3
        self.conv2 = nn.Conv1d(4, 8, 3)
        self.conv3 = nn.Conv1d(8, 16, 3)
        self.conv4 = nn.Conv1d(16, 32, 3)
        self.pool = nn.MaxPool1d(kernel_size=2)
        self.fc1 = nn.Linear(32*2, 32)
        self.fc2 = nn.Linear(32, 16)
        self.fc3 = nn.Linear(16, 2)
        self.dropout = nn.Dropout(0.5)


    def forward(self, x):
        # -> batch size, 10
        x = x.unsqueeze(1)  # batch size, 1 (channel), 10 (length of channel)
        x = self.conv1(x) # batch size, 4, (10-3)/1 + 1 = 8
        x = F.relu(x)
        x = self.dropout(x)
        x = self.conv2(x) # batch size, 8, (8-3)+1 = 6
        x = F.relu(x)
        x = self.dropout(x)
        x = self.conv3(x) #batch size, 16, (6-3)+1 = 4
        x = F.relu(x)
        x = self.dropout(x)
        x = self.conv4(x) #batch size, 32, (4-3)+1 = 2
        x = F.relu(x)
        x = self.dropout(x)
        x = x.view(-1, 32*2)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        x = F.relu(x)
        x = self.fc3(x)


        return x

In [111]:
#Topic Probabilities

class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.conv1 = nn.Conv1d(1, 4, 1) # default stride = 1
        # channels of input, channels of output, filter size: 2 x 2
        self.conv2 = nn.Conv1d(4, 8, 1)
        self.conv3 = nn.Conv1d(8, 16, 1)
        self.conv4 = nn.Conv1d(16, 32, 1)
        self.pool = nn.MaxPool1d(kernel_size=2)
        self.fc1 = nn.Linear(32*2, 32)
        self.fc2 = nn.Linear(32, 16)
        self.fc3 = nn.Linear(16, 2)
        self.dropout = nn.Dropout(0.5)


    def forward(self, x):
        # -> batch size, 10
        x = x.unsqueeze(1)  # batch size, 1 (channel), 2 (length of channel)
        x = self.conv1(x) # batch size, 4, (2-1)/1 + 1 = 2
        x = F.relu(x)
        x = self.dropout(x)
        x = self.conv2(x) # batch size, 8, (2-1)/1 + 1 = 2
        x = F.relu(x)
        x = self.dropout(x)
        x = self.conv3(x) #batch size, 16, (2-1)/1 + 1 = 2
        x = F.relu(x)
        x = self.dropout(x)
        x = self.conv4(x) #batch size, 32, (2-1)/1 + 1 = 2
        x = F.relu(x)
        x = self.dropout(x)
        x = x.view(-1, 32*2)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        x = F.relu(x)
        x = self.fc3(x)


        return x

In [120]:
#Representative Doc

class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.conv1 = nn.Conv1d(1, 4, 1) # default stride = 1
        # channels of input, channels of output, filter size: 2 x 2
        self.conv2 = nn.Conv1d(4, 8, 2)
        self.conv3 = nn.Conv1d(8, 16, 1)
        self.conv4 = nn.Conv1d(16, 32, 1)
        self.pool = nn.MaxPool1d(kernel_size=2)
        self.fc1 = nn.Linear(32*2, 32)
        self.fc2 = nn.Linear(32, 16)
        self.fc3 = nn.Linear(16, 2)
        self.dropout = nn.Dropout(0.5)


    def forward(self, x):
        # -> batch size, 10
        x = x.unsqueeze(1)  # batch size, 1 (channel), 3 (length of channel)
        x = self.conv1(x) # batch size, 4, (3-1)/1 + 1 = 3
        x = F.relu(x)
        x = self.dropout(x)
        x = self.conv2(x) # batch size, 8, (3-2)/1 + 1 = 2
        x = F.relu(x)
        x = self.dropout(x)
        x = self.conv3(x) #batch size, 16, (2-1)/1 + 1 = 2
        x = F.relu(x)
        x = self.dropout(x)
        x = self.conv4(x) #batch size, 32, (2-1)/1 + 1 = 2
        x = F.relu(x)
        x = self.dropout(x)
        x = x.view(-1, 32*2)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        x = F.relu(x)
        x = self.fc3(x)


        return x

In [101]:
#PreprocessedToken

class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.conv1 = nn.Conv1d(1, 4, 15) # default stride = 1
        # channels of input, channels of output, filter size: 2 x 2
        self.conv2 = nn.Conv1d(4, 8, 17)
        self.conv3 = nn.Conv1d(8, 16, 9)
        self.conv4 = nn.Conv1d(16, 32, 7)
        self.pool = nn.MaxPool1d(kernel_size=2)
        self.fc1 = nn.Linear(32*2, 32)
        self.fc2 = nn.Linear(32, 16)
        self.fc3 = nn.Linear(16, 2)
        self.dropout = nn.Dropout(0.5)


    def forward(self, x):
        # -> batch size, 46
        x = x.unsqueeze(1)  # batch size, 1 (channel), 8 (length of channel)
        x = self.conv1(x) # batch size, 4, (46-15)/1 + 1 = 32
        x = F.relu(x)
        x = self.dropout(x)
        x = self.conv2(x) # batch size, 8, (32-17)/1 + 1 = 16
        x = F.relu(x)
        x = self.dropout(x)
        x = self.conv3(x) #batch size, 16, (16-9)/1 + 1 = 8
        x = F.relu(x)
        x = self.dropout(x)
        x = self.conv4(x) #batch size, 32, (8-7)/1 + 1 = 2
        x = F.relu(x)
        x = self.dropout(x)
        x = x.view(-1, 32*2)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        x = F.relu(x)
        x = self.fc3(x)


        return x

In [121]:
model = ConvNet()
loss = nn.CrossEntropyLoss() #final activation function is already in here
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
n_total_steps = len(dataloader)
i = 0

#remember to set the correct data first
for epoch in range(num_epochs):
    model.train()
    for images, labels in dataloader:
        # Forward function
        outputs = model(images)
        labels= labels.long()
        l = loss(outputs, labels)
        # Backward and optimize
        optimizer.zero_grad()
        l.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()

    print (f'Epoch [{epoch+1}/{num_epochs}], Loss: {l.item():.4f}')
print('Finished Training')

Epoch [1/10], Loss: 0.6942
Epoch [2/10], Loss: 0.7000
Epoch [3/10], Loss: 0.6919
Epoch [4/10], Loss: 0.6918
Epoch [5/10], Loss: 0.6966
Epoch [6/10], Loss: 0.6943
Epoch [7/10], Loss: 0.6928
Epoch [8/10], Loss: 0.6989
Epoch [9/10], Loss: 0.6931
Epoch [10/10], Loss: 0.6936
Finished Training


In [93]:
# Evaluate the model (Topic)
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in dataloader:
        outputs = model(inputs)
        max_value, predictions = torch.max(outputs, 1)
        correct += (predictions == labels).sum().item()
        total += labels.size(0)

accuracy = correct / total
print(f'Accuracy: {accuracy:.2f}')

Accuracy: 0.51


In [113]:
# Evaluate the model (Topic Probabilities)
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in dataloader:
        outputs = model(inputs)
        max_value, predictions = torch.max(outputs, 1)
        correct += (predictions == labels).sum().item()
        total += labels.size(0)

accuracy = correct / total
print(f'Accuracy: {accuracy:.2f}')

Accuracy: 0.51


In [122]:
# Evaluate the model (Representative Documents)
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in dataloader:
        outputs = model(inputs)
        max_value, predictions = torch.max(outputs, 1)
        correct += (predictions == labels).sum().item()
        total += labels.size(0)

accuracy = correct / total
print(f'Accuracy: {accuracy:.2f}')

Accuracy: 0.49


In [103]:
# Evaluate the model (PreprocessedToken)
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in dataloader:
        outputs = model(inputs)
        max_value, predictions = torch.max(outputs, 1)
        correct += (predictions == labels).sum().item()
        total += labels.size(0)

accuracy = correct / total
print(f'Accuracy: {accuracy:.2f}')

Accuracy: 0.54


In [ ]:
torch.save(model.state_dict(), '/content/drive/MyDrive/Thesis/posts1352_standard_cnn_model.pth')

# Create a new instance of the model
model = ConvNet()

# Load the saved state dictionary
model.load_state_dict(torch.load('/content/drive/MyDrive/Thesis/posts1352_standard_cnn_model.pth'))

<All keys matched successfully>

CNN model training on CBOW embeddings

In [ ]:
import gensim

cbow_model = gensim.models.Word2Vec.load("/content/drive/MyDrive/Thesis/posts1352_standard_cbow_model")


In [ ]:
from ast import literal_eval

tokenlist = []
for x in df['PreprocessedToken']:
    tokenlist.append(' '.join(literal_eval(x)))

corpus = [token.split() for token in tokenlist]

vocab = set(cbow_model.wv.index_to_key)

In [ ]:
labels = []

for y in df['HateSpeech']:
    labels.append(y)

In [ ]:
import numpy as np

def cbow_vector(doc):
    # Remove out-of-vocabulary words
    doc = [word for word in doc if word in vocab]
    return np.mean(cbow_model.wv[doc], axis=0)

  # Transform corpus into document vectors
doc_vectors = np.array([cbow_vector(doc) for doc in corpus])

In [ ]:
# Calculate the number of full batches of size (32, 100)
batch_size = 32
num_full_batches = doc_vectors.shape[0] // batch_size

# Remove the last part that's not a full batch
doc_vectors = doc_vectors[:num_full_batches * batch_size]
labels = labels[:num_full_batches * batch_size]

# Reshape the array into batches of shape (32, 100)
batches = doc_vectors.reshape(-1, batch_size, doc_vectors.shape[1])
batched_labels = [labels[i:i + batch_size] for i in range(0, len(labels), batch_size)]

50
32


In [ ]:
class CNNConvNet(nn.Module):
    def __init__(self):
        super(CNNConvNet, self).__init__()
        self.conv1 = nn.Conv1d(1, 4, 8) # default stride = 1
        # channels of input, channels of output, filter size: 2 x 2
        self.conv2 = nn.Conv1d(4, 8, 9)
        self.conv3 = nn.Conv1d(8, 16, 6)
        self.conv4 = nn.Conv1d(16, 32, 4)
        self.pool = nn.MaxPool1d(kernel_size=2)
        self.fc1 = nn.Linear(32*2, 32)
        self.fc2 = nn.Linear(32, 16)
        self.fc3 = nn.Linear(16, 2)
        self.dropout = nn.Dropout(0.5)
        self.pooling = nn.MaxPool1d(kernel_size = 2)


    def forward(self, x):
        # -> batch size, 100
        x = x.unsqueeze(1)  # batch size, 1 (channel), 100 (length of channel)
        x = self.conv1(x) # batch size, 4, (100-9)/1 + 1 = 92
        x = F.relu(x)
        x = self.pooling(x) #batch size, 4, 92/2 = 46
        x = self.dropout(x)
        x = self.conv2(x) # batch size, 8, (46-9)+1 = 38
        x = F.relu(x)
        x = self.pooling(x) #batch size, 8, 38/2 = 19
        x = self.dropout(x)
        x = self.conv3(x) #batch size, 16, (19-6)+1 = 14
        x = F.relu(x)
        x = self.pooling(x)#batch size, 16, 14/2 = 7
        x = self.dropout(x)
        x = self.conv4(x) #batch size, 32, (7-4)+1 = 4
        x = F.relu(x)
        x = self.pooling(x) #batch size, 32, 4/2 = 2
        x = self.dropout(x)
        x = x.view(-1, 32*2)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        x = F.relu(x)
        x = self.fc3(x)


        return x

In [ ]:
model = CNNConvNet()
loss = nn.CrossEntropyLoss() #final activatioh function is already in here
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
i = 0

for epoch in range(num_epochs):
    model.train()
    for batch , labels in zip(batches, batched_labels):
        # Forward function
        batch = torch.from_numpy(batch)
        outputs = model(batch)
        labels = torch.tensor(labels)
        l = loss(outputs, labels)
        # Backward and optimize
        optimizer.zero_grad()
        l.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()

    print (f'Epoch [{epoch+1}/{num_epochs}], Loss: {l.item():.4f}')
print('Finished Training')

Epoch [1/10], Loss: 0.7392
Epoch [2/10], Loss: 0.7249
Epoch [3/10], Loss: 0.7159
Epoch [4/10], Loss: 0.7103
Epoch [5/10], Loss: 0.7068
Epoch [6/10], Loss: 0.7037
Epoch [7/10], Loss: 0.7018
Epoch [8/10], Loss: 0.7000
Epoch [9/10], Loss: 0.6981
Epoch [10/10], Loss: 0.6980
Finished Training


In [ ]:
# Evaluate the model (CBOW)
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for batch , labels in zip(batches, batched_labels):
        batch = torch.from_numpy(batch)
        outputs = model(batch)
        labels = torch.tensor(labels)
        max_value, predictions = torch.max(outputs, 1)
        correct += (predictions == labels).sum().item()
        total += labels.size(0)

accuracy = correct / total
print(f'Accuracy: {accuracy:.2f}')

Accuracy: 0.49


In [ ]:
torch.save(model.state_dict(), '/content/drive/MyDrive/Thesis/posts1352_standard_cbow_cnn_model.pth')

# Create a new instance of the model
model = CNNConvNet()

# Load the saved state dictionary
model.load_state_dict(torch.load('/content/drive/MyDrive/Thesis/posts1352_standard_cbow_cnn_model.pth'))

<All keys matched successfully>